## Imports

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout

files = os.listdir(path='data/')

## Types of recordings of Activities

In [2]:
devices = ["Respeck", "Thingy"]
student_numbers = ["s1923862", "s1915704", "s1971889"]
eating_static_activities = ["Sitting", "Standing"]
static_activities = ["Lying down right", "Lying down on left", "Lying down on stomach", "Lying down back"]
dynamic_activities = ["Shuffle walking", "Normal walking", "Running", "Ascending stairs", "Descending stairs", "Miscellaneous movements"]
sub_activities = ["Normal", "Talking", "Singing", "Coughing", "Laughing", "Hyperventilating"]
eating_sub_activity = "Eating"

f"{devices[0]}_{student_numbers[0]}_{static_activities[0]}_{sub_activities[0]}.csv"

'Respeck_s1923862_Lying down right_Normal.csv'

## Build consolidated DF of all recordings

In [3]:
def get_new_dataframe(student_number, activity, sub_activity, is_static):
    file = f"{devices[0]}_{student_number}_{activity}_{sub_activity}.csv"
    filename = "data/"+ file
    df = pd.read_csv(filename, index_col='Unnamed: 0')
    df['student_number'] = student_number
    df['activity'] = activity
    df['sub_activity'] = sub_activity
    df['is_static'] = is_static
    return df.drop('ind', axis=1)

dataset = pd.DataFrame()

for student_number in student_numbers:
    for activity in static_activities:
        for sub_activity in sub_activities:
            dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, sub_activity, 1)])

    for activity in eating_static_activities:
        for sub_activity in sub_activities:
            dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, sub_activity, 1)])
        dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, eating_sub_activity, 1)])

    for activity in dynamic_activities:
        dataset = pd.concat([dataset, get_new_dataframe(student_number, activity, sub_activities[0], 0)])

dataset.head(10)

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,student_number,activity,sub_activity,is_static
0,1695993500435,0.866455,0.200378,0.483093,0.171875,-0.171875,0.093750,s1923862,Lying down right,Normal,1
1,1695993500475,0.865967,0.199890,0.478699,-0.109375,-0.062500,-0.125000,s1923862,Lying down right,Normal,1
2,1695993500515,0.861816,0.194519,0.488220,-0.046875,-0.390625,-0.156250,s1923862,Lying down right,Normal,1
3,1695993500555,0.869629,0.199158,0.488464,-0.109375,0.187500,0.109375,s1923862,Lying down right,Normal,1
4,1695993500595,0.862061,0.198181,0.488464,-0.093750,0.515625,0.218750,s1923862,Lying down right,Normal,1
5,1695993500635,0.864014,0.198669,0.485291,-0.218750,0.562500,0.156250,s1923862,Lying down right,Normal,1
6,1695993500675,0.865234,0.199158,0.476990,-0.140625,0.453125,0.203125,s1923862,Lying down right,Normal,1
7,1695993500715,0.869629,0.190857,0.488708,-1.546875,0.812500,0.062500,s1923862,Lying down right,Normal,1
8,1695993500755,0.861572,0.193542,0.485046,-0.140625,0.953125,0.265625,s1923862,Lying down right,Normal,1
9,1695993500795,0.866455,0.199402,0.478210,-0.546875,0.984375,0.343750,s1923862,Lying down right,Normal,1


## Sliding window to generate overlapping intervals of recordings
Interval size is temporarily set to 2 secs (ie, 50 recording points)

In [4]:
random_seed = 42   
n_time_steps = 50 
n_features = 3
step = 10
n_classes = 6 
n_epochs = 60       
batch_size = 1024   
learning_rate = 0.0025
l2_loss = 0.0015
segments = []
labels = []

x = 760
while x<dataset.shape[0]:
    for i in range(x-760,  x-n_time_steps, step):  
        accel_xs = dataset['accel_x'].values[i: i + n_time_steps]
        accel_ys = dataset['accel_y'].values[i: i + n_time_steps]
        accel_zs = dataset['accel_z'].values[i: i + n_time_steps]

        label = dataset['activity'].values[i]

        segments.append([accel_xs, accel_ys, accel_zs])

        labels.append(label)
    
    x+=760

#reshape the segments which is (list of arrays) to a list
reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)


labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

X_train, X_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size = 0.2, random_state = random_seed)


## Build Multi-layer model
Modify hyperparameters

In [5]:
model = Sequential()
# RNN layer
model.add(LSTM(units = 128, input_shape = (X_train.shape[1], X_train.shape[2])))
# # Dropout layer
model.add(Dropout(0.5))
# Dense layer with ReLu
model.add(Dense(units = 64, activation='relu'))
# Softmax layer
model.add(Dense(y_train.shape[1], activation = 'softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               67584     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 12)                780       
                                                                 
Total params: 76,620
Trainable params: 76,620
Non-trainable params: 0
_________________________________________________________________


## Train model

In [7]:
model.fit(X_train, y_train, epochs = 10, validation_split = 0.20, batch_size = batch_size, verbose = 1)
model.evaluate(X_test, y_test, batch_size = batch_size, verbose = 1)

Epoch 1/10
6/6 [==============================] - 3s 441ms/step - loss: 0.3982 - accuracy: 0.8454 - val_loss: 0.3407 - val_accuracy: 0.8683
Epoch 2/10
6/6 [==============================] - 3s 447ms/step - loss: 0.3810 - accuracy: 0.8538 - val_loss: 0.3419 - val_accuracy: 0.8716
Epoch 3/10
6/6 [==============================] - 3s 448ms/step - loss: 0.3894 - accuracy: 0.8495 - val_loss: 0.3439 - val_accuracy: 0.8636
Epoch 4/10
6/6 [==============================] - 3s 439ms/step - loss: 0.3898 - accuracy: 0.8486 - val_loss: 0.3618 - val_accuracy: 0.8515
Epoch 5/10
6/6 [==============================] - 3s 459ms/step - loss: 0.3926 - accuracy: 0.8427 - val_loss: 0.3660 - val_accuracy: 0.8569
Epoch 6/10
6/6 [==============================] - 3s 460ms/step - loss: 0.3941 - accuracy: 0.8451 - val_loss: 0.3440 - val_accuracy: 0.8663
Epoch 7/10
6/6 [==============================] - 3s 472ms/step - loss: 0.3821 - accuracy: 0.8520 - val_loss: 0.3480 - val_accuracy: 0.8629
Epoch 8/10
6/6 [====

[0.3733690083026886, 0.8597528338432312]

The following cell is how we expect to be able to use the model to predict the activity

In [10]:
# How to predict activity
activities = ['Ascending stairs',
 'Descending stairs',
 'Lying down back',
 'Lying down on left',
 'Lying down on stomach',
 'Lying down right',
 'Miscellaneous movements',
 'Normal walking',
 'Running',
 'Shuffle walking',
 'Sitting',
 'Standing']

y_prd = model.predict(X_test)
activities[np.argmax(y_prd[0])]

59/59 [==============================] - 1s 16ms/step


'Standing'

In [11]:
confusion_matrix(list(map(np.argmax, y_test)), list(map(np.argmax, y_prd)))

array([[ 50,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   0],
       [  0,  12,   0,   0,   0,   0,   0,   0,   0,   0,  31,  14],
       [  0,   0, 249,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 248,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 257,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 244,   0,   0,   0,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,  10,   0,   8,   0,   6,   4],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  41,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,  50,   0,   0,   0],
       [  5,   0,   0,   0,   0,   0,   0,   0,   0,   0,  34,   4],
       [ 16,   0,   0,   0,   0,   0,   0,   0,   0,   0, 235,  54],
       [  0,   0,  19,   0,   0,   0,   0,   0,   0,   0,  18, 245]],
      dtype=int64)

## Export trained model to .tflite file

In [13]:
from tensorflow import lite
import tensorflow as tf
converter = lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('lstm2.tflite', 'wb').write(tfmodel)


INFO:tensorflow:Assets written to: C:\Users\Arun8\AppData\Local\Temp\tmp645httjo\assets


INFO:tensorflow:Assets written to: C:\Users\Arun8\AppData\Local\Temp\tmp645httjo\assets


92552

In [14]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="lstm.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
# input_data = np.zeros((3, 50),dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[-4.727551   -4.010392   -3.9730744  -3.9590197  -3.6293461  -3.9232976
  -4.2992415  -3.8785055  -3.075963   -5.2122164  -4.2707667  -4.754139
  -4.840304   -4.429952   -4.1886954  -4.5068417  -4.60805    -3.8054304
  -3.8760874  -4.3564243  -3.8211055  -3.4086878  -3.631978   -3.2071946
  -3.9590394  -4.617197   -4.70587    -3.9188297  -4.344318   -4.5759325
  -5.261177   -3.0069904  -3.8978975  -4.447899   -4.294748   -4.7112155
  -4.5429325  -4.2497783  -3.5728066  -4.703055   -4.4069805  -4.5004325
  -3.8658457  -4.7675667  -4.260559   -4.4982905  -4.193301   -4.5409875
  -4.9150705  -5.068224   -4.7765284  -5.5043483  -4.7737193  -3.786916
  -4.1793213  -4.7608953  -3.9973407  -5.12015    -4.6868668  -1.9843068
  -2.4742975  -4.579955   -4.6514096  -3.5777776  -3.9996676  -3.9738598
  -4.5804973  -4.009182   -4.179177   -3.9049804  -4.3946495  -3.32036
  -4.734847   -2.862324   -4.9822745  -3.982811   -4.2214246  -3.9363012
  -2.8993793  -3.3020766  -4.17677    -4.5623384  -4.51

In [16]:
input_data

array([[[[0.37276003, 0.9727126 , 0.1590675 ]]]], dtype=float32)

In [ ]:
output_data.shape

In [ ]:
output_data

In [ ]:
input_data.shape

In [ ]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="model3.tflite")
interpreter.allocate_tensors()

# Print input shape and type
inputs = interpreter.get_input_details()
print('{} input(s):'.format(len(inputs)))
for i in range(0, len(inputs)):
    print('{} {}'.format(inputs[i]['shape'], inputs[i]['dtype']))

# Print output shape and type
outputs = interpreter.get_output_details()
print('\n{} output(s):'.format(len(outputs)))
for i in range(0, len(outputs)):
    print('{} {}'.format(outputs[i]['shape'], outputs[i]['dtype']))

In [17]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="linear.tflite")
interpreter.allocate_tensors()

# Print input shape and type
inputs = interpreter.get_input_details()
print('{} input(s):'.format(len(inputs)))
for i in range(0, len(inputs)):
    print('{} {}'.format(inputs[i]['shape'], inputs[i]['dtype']))

# Print output shape and type
outputs = interpreter.get_output_details()
print('\n{} output(s):'.format(len(outputs)))
for i in range(0, len(outputs)):
    print('{} {}'.format(outputs[i]['shape'], outputs[i]['dtype']))

1 input(s):
[1 1] <class 'numpy.float32'>

1 output(s):
[1 1] <class 'numpy.float32'>


In [19]:
model.save("ourLstm.pb")

INFO:tensorflow:Assets written to: ourLstm.pb\assets


INFO:tensorflow:Assets written to: ourLstm.pb\assets


In [20]:
converter=tf.lite.TFLiteConverter.from_saved_model(r"C:\Users\Arun8\OneDrive\Desktop\PDIoT\CW3-common-app\pdiot-HAR-model\ourLstm.pb")

In [22]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model=converter.convert()
open("lstm2.tflite","wb").write(tflite_model)

92608

In [23]:
input_shape


array([1, 1, 1, 3])